In [1]:
import soundfile as sf
from datasets import load_dataset
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel

In [2]:
model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

Config of the decoder: <class 'transformers.models.speech_to_text_2.modeling_speech_to_text_2.Speech2Text2ForCausalLM'> is overwritten by shared decoder config: Speech2Text2Config {
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_cross_attention": true,
  "architectures": [
    "Speech2TextForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "conv_channels": 1024,
  "conv_kernel_sizes": [
    5,
    5
  ],
  "d_model": 256,
  "decoder_attention_heads": 4,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 7,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 4,
  "encoder_ffn_dim": 2048,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "input_channels": 1,
  "input_feat_per_channel": 80,
  "is_decoder": true,
  "is_encoder_decoder": true,
  "max_length"

In [10]:
inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
print(inputs['input_values'].shape)

generated_ids = model.generate(inputs=inputs["input_values"], attention_mask=inputs["attention_mask"])
print(generated_ids)


transcription = processor.batch_decode(generated_ids)
print(transcription)

torch.Size([1, 93680])
tensor([[   2, 1845,  678,  287,  129,   11,    6,  114, 5364,  262,    6, 1267,
          106, 2449,    7,  242,   40, 6879,    5,   84, 2623,  348,  143,   54,
           15, 1382,  888,  893,    4,    2]])
['</s> Herr Quilter ist der Apostel der Mittelschicht und wir sind froh, sein Evangelium zu begrüßen. </s>']
